# 0. Imports

In [1]:
import pandas as pd
import psycopg2 as pg

import credentials

# 1. Load Data

In [4]:
sql_query = """
select 
  u.id,
  u.gender,
  u.age,
  u.region_code,
  u.policy_sales_channel,
  i.previously_insured,
  i.annual_premium,
  i.vintage,
  v.driving_license,
  v.vehicle_age,
  v.vehicle_damage,
  i.response
from pa004.users u 
inner join pa004.insurance i on u.id = i.id 
inner join pa004.vehicle v on u.id = v.id
where u.id between 1 and 381109 -- limit id to guarantee the same data is fetched
"""

conn = pg.connect(
    user=credentials.USER,
    password=credentials.PWD,
    host=credentials.HOST,
    port=credentials.PORT,
    database=credentials.DB
)

df_raw = pd.read_sql(sql_query, conn)

# 2. Data Description

In [27]:
df_dd = df_raw.copy()

## 2.1. Data Dimension

In [28]:
print(f'Number of rows: {df_dd.shape[0]}')
print(f'Number of columns: {df_dd.shape[1]}')

Number of rows: 381109
Number of columns: 12


## 2.2. Data Types

In [29]:
df_dd.sample()

,id,gender,age,region_code,policy_sales_channel,previously_insured,annual_premium,vintage,driving_license,vehicle_age,vehicle_damage,response
150180,57405,Female,22,17.0,152.0,1,64339.0,214,1,< 1 Year,No,0


In [30]:
df_dd.dtypes

id                        int64
gender                   object
age                       int64
region_code             float64
policy_sales_channel    float64
previously_insured        int64
annual_premium          float64
vintage                   int64
driving_license           int64
vehicle_age              object
vehicle_damage           object
response                  int64
dtype: object

In [34]:
# change the feature vehicle_damage to int - Yes = 1 and No = 0
mapping_dict = {'Yes': 1, 'No': 0}
df_dd['vehicle_damage'] = df_dd['vehicle_damage'].map(mapping_dict)

# change the feature gender to int - Female = 1 and Male = 0
mapping_dict = {'Female': 1, 'Male': 0}
df_dd['gender'] = df_dd['gender'].map(mapping_dict)

df_dd.dtypes

id                        int64
gender                    int64
age                       int64
region_code             float64
policy_sales_channel    float64
previously_insured        int64
annual_premium          float64
vintage                   int64
driving_license           int64
vehicle_age              object
vehicle_damage          float64
response                  int64
dtype: object

## 2.3. Check NAs

In [32]:
df_dd.isna().sum()

id                      0
gender                  0
age                     0
region_code             0
policy_sales_channel    0
previously_insured      0
annual_premium          0
vintage                 0
driving_license         0
vehicle_age             0
vehicle_damage          0
response                0
dtype: int64

No NAs found, no further action needed.

## 2.4. Descriptive Statistics

In [35]:
# lets separate the features in numerical and categorical
num_features = ['age', 'region_code', 'policy_sales_channel', 'annual_premium', 'vintage', 'response']
cat_features = ['gender', 'region_code', 'policy_sales_channel', 'previously_insured', 'driving_license', 'vehicle_age', 'vehicle_damage']